# Projects

> Create and load pre-built aimagic notebooks for contexts

In [ ]:
#| default_exp projects

In [ ]:

import json
from pathlib import Path
import ContextKit.contexts as ctx
from fastcore.all import bind, call_parse

from typing import Annotated
import sys

In [ ]:

def create_cell(cell_type: str, outputs: list, source: list):
    assert cell_type in ('code','markdown')
    # source must be split on newline - check execnb
    cell = {'cell_type': cell_type, 'execution_count': None, 'metadata': {}, 'source': source} 
    if cell_type == 'code': cell['outputs'] = outputs
    return cell

In [ ]:

def create_magic_template_cells(ctx_name):
    _fn = bind(create_cell,outputs=[], cell_type='code')  
    _ctx_dict = getattr(globals()['ctx'], templates[ctx_name], None)()
    cells = [_fn(source=[f"from ContextKit.contexts import *"]),
            _fn(source=[f"{templates[ctx_name]} = {templates[ctx_name]}()\n"] + [f"{k} = {templates[ctx_name]}['{k}']\n" for k in _ctx_dict.keys()]),
            _fn(source=[f"%%ai -c\nPlease read {', '.join([f'$`{k}`' for k in _ctx_dict.keys()])} and give a brief description of what each includes."]),]
    return cells

In [ ]:

def create_nb(cells, path):
    assert isinstance(cells, list)
    nb = {"cells": cells, "metadata": {}, "nbformat": 4, "nbformat_minor": 4}
    Path(path).write_text(json.dumps(nb, indent=2))

In [ ]:

def create_magic_template_nb(ctx_name, path):
    create_nb(create_magic_template_cells(ctx_name), path=path)

In [ ]:

templates = {name[4:]: name for name in dir(ctx) if name.startswith('ctx_')}

In [ ]:
create_magic_template_nb('fasthtml','../testnb2.ipynb')

dict_keys(['fasthtml_llms-ctx', 'hypermedia_summary'])
